In [ ]:
import numpy as np
from scipy.stats import chi2, truncnorm

In [ ]:
def rand_constrained_Wish(Psi0, nu, h): 
    dm = Psi0.shape[0]
    zeroidx = np.array([i for i in range(len(h)) if h[i] == 0])
    nonzeroidx = np.setdiff1d(np.arange(dm), zeroidx)
    if zeroidx.size == dm:
        return np.eye(dm) * nu
    elif zeroidx.size == 0:
        subh = h
        subPsi0 = Psi0
    else:
        subh = np.array([h[i] for i in nonzeroidx])
        subPsi0 = Psi0[np.ix_(nonzeroidx, nonzeroidx)]

    subdm = subPsi0.shape[0]
    U = np.linalg.cholesky(subPsi0).T
    A = np.zeros((subdm, subdm))
    for i in range(subdm):
        A[i, i] = np.sqrt(chi2.rvs(nu - i + 1))
    npairs = int(dm * (dm - 1) / 2)
    dimpairs = np.zeros((npairs, 2), dtype=int)
    counter = 0
    for k in range(1, dm):
        for i in range(1, k + 1):
            dimpairs[counter, :] = [i, k + 1]
            counter += 1
        
    if zeroidx.size > 0:
        rmpairs = np.nonzero(np.any(np.isin(dimpairs-1, zeroidx), axis=1))[0] #!!!!!!!!!!!!!!!!
        dimpairs = np.delete(dimpairs, rmpairs, axis=0)
        npairs = dimpairs.shape[0]
    
    for idx in range(npairs):
        m, p = dimpairs[idx]
        mstar = np.where(nonzeroidx+1 == m)[0][0] #!!!!!!!!!!!!!
        pstar = np.where(nonzeroidx+1 == p)[0][0] #!!!!!!!!!!!!!
        if mstar == 0:
            premult = U[mstar, mstar] * A[mstar, mstar]
            term4 = 0.0
            for i in range(mstar, pstar):
                term4 += U[i, pstar] * A[i, mstar]

            if np.sign(h[m-1] * h[p-1]) == -1:
                lb = -np.inf
                ub = -term4 / U[pstar, pstar]
            else:
                lb = -term4 / U[pstar, pstar]
                ub = np.inf

            A[pstar, mstar] = truncnorm.rvs(lb, ub)
        else:
            term1 = 0.0
            for k in range(mstar):
                outerterm = 0.0
                for j in range(k + 1):
                    innerterm = 0.0
                    for i in range(j, pstar + 1):
                        innerterm += A[i, j] * U[i, pstar]
                    outerterm += innerterm * A[k, j]
                term1 += U[k, mstar] * outerterm

            outerterm2 = 0.0
            for j in range(mstar):
                innerterm2 = 0.0
                for i in range(j, pstar + 1):
                    innerterm2 += A[i, j] * U[i, pstar]
                outerterm2 += innerterm2 * A[mstar, j]
            term2 = U[mstar, mstar] * outerterm2

            term3 = 0.0
            for i in range(mstar, pstar):
                term3 += A[i, mstar] * U[i, pstar]
            term3 *= A[mstar, mstar] * U[mstar, mstar]

            if np.sign(h[m-1] * h[p-1]) == -1:
                lb = -np.inf
                ub = (-term1 - term2 - term3) / (U[pstar, pstar] * U[mstar, mstar] * A[mstar, mstar])
            else:
                lb = (-term1 - term2 - term3) / (U[pstar, pstar] * U[mstar, mstar] * A[mstar, mstar])
                ub = np.inf

            A[pstar, mstar] = truncnorm.rvs(lb, ub)

    if len(zeroidx) == 0:
        return U.T @ A @ A.T @ U
    else:
        matmult = U.T @ A @ A.T @ U
        out = np.eye(dm)
        out[np.ix_(nonzeroidx, nonzeroidx)] = matmult

        for zz in zeroidx:
            out[zz, zz] = nu
        return out

In [ ]:
def rand_constrained_MVN(Sigma, mu0, h):
    dm = Sigma.shape[0]
    zeroidx = np.where(h == 0)[0]
    nonzeroidx = np.setdiff1d(np.arange(dm), zeroidx)

    if zeroidx.size == dm:
        return np.zeros(dm)
    elif zeroidx.size == 0:
        subh = h
        subSigma = Sigma
        submu0 = mu0
    else:
        subh = h[nonzeroidx]
        subSigma = Sigma[np.ix_(nonzeroidx, nonzeroidx)]
        submu0 = mu0[nonzeroidx]

    subdm = subSigma.shape[0]
    A = np.linalg.cholesky(subSigma)

    z = np.zeros(subdm)
    for i in range(subdm):
        bound = -mu0[i]
        if i == 0:
            bound /= A[i, i]
        else:
            bound = (bound - np.dot(A[i, :i].T, z[:i])) / A[i, i]

        if subh[i] == 1:
            z[i] = truncnorm.rvs(bound, np.inf)
        else:
            z[i] = truncnorm.rvs(-np.inf, bound)

    subout = np.dot(A, z) + submu0
    out = np.zeros(dm)
    out[nonzeroidx] = subout
    return out